In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)

def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
print("ok1")

ok1


In [4]:
j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

In [2]:
print(H6(5,3));print(type(H6(5,3)))

-3647/35664401793024*x^5 - 7399/123834728448*x^4 - 427/17915904*x^3 - 77/13824*x^2 - 7/24*x + 1
<class 'sage.rings.polynomial.polynomial_rational_flint.Polynomial_rational_flint'>


In [15]:
print(H6strike(5,3))

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1


In [3]:
H6(5,4)

-315/34359738368*x^5 - 185/268435456*x^4 - 35/262144*x^3 - 25/4096*x^2 - 5/16*x + 1

In [4]:
H6(5,3)

-3647/35664401793024*x^5 - 7399/123834728448*x^4 - 427/17915904*x^3 - 77/13824*x^2 - 7/24*x + 1

In [18]:
poly=H6strike(5,3)
print(poly)
for k in [1..5]:
    print([k,-504*sigma(k,5)])

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1
[1, -504]
[2, -16632]
[3, -122976]
[4, -532728]
[5, -1575504]


In [3]:
data=[]
no=[]
import time
startall=time.time()
for m in [3..603]:
    poly=H6strike(200,m)
    pd=poly.degree()
    if pd!=200:no=no+[m]
    data=data+[[m,poly]]
    finish=time.time()
    print([m,pd,(finish-startall)/60])
import pickle
wfile = open('/Users/barrybrent/7sept22no2.txt','wb') # H6strike series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()
print("no:")
print(no)

[3, 200, 0.3228157639503479]
[4, 200, 0.5395832657814026]
[5, 200, 0.9730966289838155]
[6, 200, 1.1737838427225749]
[7, 200, 1.6637703458468118]
[8, 200, 2.0438918987909953]
[9, 200, 2.587768228848775]
[10, 200, 2.925911509990692]
[11, 200, 3.4601980129877727]
[12, 200, 3.9202319979667664]
[13, 200, 4.503910263379415]
[14, 200, 4.876205062866211]
[15, 200, 5.528043460845947]
[16, 200, 5.976241878668467]
[17, 200, 6.59403729836146]
[18, 200, 7.003653979301452]
[19, 200, 7.603173863887787]
[20, 200, 8.145133010546367]
[21, 200, 8.818001846472422]
[22, 200, 9.225763245423634]
[23, 200, 9.84667636156082]
[24, 200, 10.413873879114787]
[25, 200, 11.078513765335083]
[26, 200, 11.540729530652364]
[27, 200, 12.211520377794901]
[28, 200, 12.782757262388865]
[29, 200, 13.416395111878712]
[30, 200, 13.913124211629231]
[31, 200, 14.544982345898946]
[32, 200, 15.064803345998127]
[33, 200, 15.79735626379649]
[34, 200, 16.26790851354599]
[35, 200, 17.017333710193633]
[36, 200, 17.63767342567444]
[37, 

[275, 200, 267.9788179477056]
[276, 200, 268.883393462499]
[277, 200, 269.7933107932409]
[278, 200, 270.49742229382196]
[279, 200, 271.4757163445155]
[280, 200, 272.37489622831345]
[281, 200, 273.2732900818189]
[282, 200, 274.04359259605405]
[283, 200, 274.97058601379393]
[284, 200, 275.8022179325422]
[285, 200, 276.8440264821053]
[286, 200, 277.5788662632306]
[287, 200, 278.5734765648842]
[288, 200, 279.4063299934069]
[289, 200, 280.3747618794441]
[290, 200, 281.1296517133713]
[291, 200, 282.1033937652906]
[292, 200, 282.9075590292613]
[293, 200, 283.8405749599139]
[294, 200, 284.6162734429042]
[295, 200, 285.58243862787884]
[296, 200, 286.40211764574053]
[297, 200, 287.3733853141467]
[298, 200, 288.0543092449506]
[299, 200, 289.0111819108327]
[300, 200, 289.90722823143005]
[301, 200, 290.9007254997889]
[302, 200, 291.5860144615173]
[303, 200, 292.5682026942571]
[304, 200, 293.3835139592489]
[305, 200, 294.35538791418077]
[306, 200, 295.10927024284996]
[307, 200, 296.03911286592484]
[

[545, 200, 511.24890238046646]
[546, 200, 512.091617679596]
[547, 200, 513.0326921105385]
[548, 200, 513.9248588959376]
[549, 200, 514.9701775471369]
[550, 200, 515.8080139279366]
[551, 200, 516.8059313774108]
[552, 200, 517.7338009953498]
[553, 200, 518.7315692305565]
[554, 200, 519.4810039440791]
[555, 200, 520.5686762491862]
[556, 200, 521.4236378947894]
[557, 200, 522.419655529658]
[558, 200, 523.2261498769125]
[559, 200, 524.2443687438965]
[560, 200, 525.1781713763872]
[561, 200, 526.2536552111308]
[562, 200, 527.0015952467918]
[563, 200, 527.9703559438387]
[564, 200, 528.9146485447884]
[565, 200, 529.9291731476784]
[566, 200, 530.7105949123701]
[567, 200, 531.780875146389]
[568, 200, 532.6591953317325]
[569, 200, 533.5964023272196]
[570, 200, 534.4536554773648]
[571, 200, 535.418603392442]
[572, 200, 536.3285973946254]
[573, 200, 537.3590871771177]
[574, 200, 538.1531083623569]
[575, 200, 539.2069423993429]
[576, 200, 540.0991304636002]
[577, 200, 541.0787876645724]
[578, 200, 54